# Task 1: Model Training

# 1. Set Up the Environment:

- Install necessary libraries such as PyTorch and torchvision.
- Import required packages

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
from torchvision import models



Files already downloaded and verified
Files already downloaded and verified


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 160MB/s]


[Epoch 1, Batch 200] loss: 1.226
Accuracy of the network on the validation images after epoch 1: 70.34%
[Epoch 2, Batch 200] loss: 0.783
Accuracy of the network on the validation images after epoch 2: 75.76%
[Epoch 3, Batch 200] loss: 0.668
Accuracy of the network on the validation images after epoch 3: 77.43%
[Epoch 4, Batch 200] loss: 0.594
Accuracy of the network on the validation images after epoch 4: 77.54%
[Epoch 5, Batch 200] loss: 0.562
Accuracy of the network on the validation images after epoch 5: 79.57%
[Epoch 6, Batch 200] loss: 0.515
Accuracy of the network on the validation images after epoch 6: 79.96%
[Epoch 7, Batch 200] loss: 0.482
Accuracy of the network on the validation images after epoch 7: 79.10%
[Epoch 8, Batch 200] loss: 0.467
Accuracy of the network on the validation images after epoch 8: 79.50%
[Epoch 9, Batch 200] loss: 0.442
Accuracy of the network on the validation images after epoch 9: 81.54%
[Epoch 10, Batch 200] loss: 0.428
Accuracy of the network on the

# 2. Download and Prepare the CIFAR-10 Dataset:

- Download the CIFAR-10 dataset using torchvision.datasets.
- Split the dataset into training (40,000 images) and validation (10,000 images) sets.

In [ ]:
# Define the transformations for the training and validation datasets
transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

# Download and load the CIFAR-10 training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)

# Download and load the CIFAR-10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)

# Split the training dataset into training (40,000 images) and validation (10,000 images) sets
train_size = 40000
val_size = 10000
train_dataset, val_dataset = random_split(trainset, [train_size, val_size])

# Define batch size
batch_size = 128

# Create data loaders for training, validation, and test datasets
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)



# 3. Define the CNN Model:¶

- Choose a CNN architecture (Resnet18).

- Modify the last layer to have 10 output classes for the CIFAR-10 dataset.

In [ ]:
# Load the pre-trained ResNet18 model
net = models.resnet18(pretrained=True)

# Modify the last layer to match the number of classes in CIFAR-10
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 10)

# 4. Define Loss Function and Optimizer:

- Use CrossEntropyLoss and an optimizer like SGD

In [ ]:


# Check if GPU is available and use it
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# 5. Train the Model:

- Train the model for 10 epochs and evaluate on the validation set.

In [ ]:


# Number of epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    net.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

   

# 6. Evaluate on the Test Set:

- Report the accuracy on the test dataset

In [ ]:
 # Evaluate on validation data after each epoch
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the validation images after epoch {epoch + 1}: {100 * correct / total:.2f}%')

print('Finished Training')


# Task 2: Model Pruning


## 1. Apply Pruning Techniques & Evaluate Pruned Models:

- Use PyTorch's pruning functionalities to prune the model.
- Experiment with different pruning ratios.
- Evaluate the pruned models on the validation set

In [3]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
from torchvision.models import resnet18, ResNet18_Weights

In [4]:
# Function to apply pruning to the model
def apply_pruning(model, amount):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
    return model

In [ ]:
# Function to remove the pruning reparameterization
def remove_pruning_reparameterization(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            try:
                prune.remove(module, 'weight')
            except ValueError:
                continue
    return model

In [ ]:
# Function to count the number of unmasked (non-zero) weights
def count_unmasked_weights(model):
    unmasked_weights = 0
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            if hasattr(module, 'weight_mask'):
                unmasked_weights += module.weight_mask.sum().item()
            else:
                unmasked_weights += module.weight.numel()
    return unmasked_weights

# Function to count the number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)



In [ ]:
# Function to evaluate the model
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

## 2

In [ ]:
# Initialize the model and modify the final layer
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model.to(device)

# Load the trained model state dictionary
model.load_state_dict(torch.load('original_model.pth'))

# Evaluate the trained model
original_accuracy = evaluate_model(model, valloader)
print(f'Accuracy of the original model: {original_accuracy:.2f}%')

# Save the trained model state dictionary
torch.save(model.state_dict(), 'original_model.pth')

## 3.  

In [7]:

# Apply pruning with multiple ratios
pruning_ratios = [0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9]
pruned_models = {}
original_num_params = count_parameters(model)
original_num_unmasked_weights = count_unmasked_weights(model)
print(f'Original number of parameters: {original_num_params}')
print(f'Original number of unmasked weights: {original_num_unmasked_weights}')

for ratio in pruning_ratios:
    # Create a new model instance and load the trained state dictionary
    model_copy = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    num_ftrs = model_copy.fc.in_features
    model_copy.fc = nn.Linear(num_ftrs, 10)
    model_copy.to(device)
    
    model_copy.load_state_dict(torch.load('original_model.pth'))  # Load the trained model

    # Apply pruning
    pruned_model = apply_pruning(model_copy, ratio)
    
    # Count the number of unmasked weights
    num_unmasked_weights = count_unmasked_weights(pruned_model)
    
    # Remove reparameterization
    pruned_model = remove_pruning_reparameterization(pruned_model)
    
    pruned_models[ratio] = pruned_model
    
    # Display number of parameters and number of unmasked weights
    num_params = count_parameters(pruned_model)
    print(f'Pruning ratio: {ratio}')
    print(f'Number of parameters: {num_params}')
    print(f'Number of unmasked weights: {num_unmasked_weights}')

# Evaluate the pruned models
for ratio, model in pruned_models.items():
    accuracy = evaluate_model(model, valloader)
    print(f'Accuracy of the pruned model with ratio {ratio}: {accuracy:.2f}%')

Accuracy of the original model: 85.48%
Original number of parameters: 11181642
Original number of unmasked weights: 11172032
Pruning ratio: 0.2
Number of parameters: 11181642
Number of unmasked weights: 8937625.0
Pruning ratio: 0.3
Number of parameters: 11181642
Number of unmasked weights: 7820423.0
Pruning ratio: 0.4
Number of parameters: 11181642
Number of unmasked weights: 6703219.0
Pruning ratio: 0.6
Number of parameters: 11181642
Number of unmasked weights: 4468813.0
Pruning ratio: 0.7
Number of parameters: 11181642
Number of unmasked weights: 3351609.0
Pruning ratio: 0.8
Number of parameters: 11181642
Number of unmasked weights: 2234407.0
Pruning ratio: 0.9
Number of parameters: 11181642
Number of unmasked weights: 1117203.0
Accuracy of the pruned model with ratio 0.2: 85.07%
Accuracy of the pruned model with ratio 0.3: 85.03%
Accuracy of the pruned model with ratio 0.4: 84.21%
Accuracy of the pruned model with ratio 0.6: 77.24%
Accuracy of the pruned model with ratio 0.7: 53.36%

In [8]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
from torchvision.models import resnet18, ResNet18_Weights

# Initialize the model and modify the final layer
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model.to(device)

# Load the trained model state dictionary
model.load_state_dict(torch.load('original_model.pth'))

# Evaluate the trained model
original_accuracy = evaluate_model(model, valloader)
print(f'Accuracy of the original model: {original_accuracy:.2f}%')

# Set the target accuracy
target_accuracy = original_accuracy - 1.0

# Apply pruning with refined ratios between 0.25 and 0.5
pruning_ratios = [0.25 + 0.01 * i for i in range(26)]  # Pruning ratios from 0.25 to 0.5
best_ratio = 0
best_accuracy = 0
for ratio in pruning_ratios:
    # Create a new model instance and load the trained state dictionary
    model_copy = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    num_ftrs = model_copy.fc.in_features
    model_copy.fc = nn.Linear(num_ftrs, 10)
    model_copy.to(device)
    
    model_copy.load_state_dict(torch.load('original_model.pth'))  # Load the trained model

    # Apply pruning
    pruned_model = apply_pruning(model_copy, ratio)
    
    # Count the number of unmasked weights
    num_unmasked_weights = count_unmasked_weights(pruned_model)
    
    # Remove reparameterization
    pruned_model = remove_pruning_reparameterization(pruned_model)
    
    # Evaluate the pruned model
    accuracy = evaluate_model(pruned_model, valloader)
    print(f'Pruning ratio: {ratio}, Accuracy: {accuracy:.2f}%, Unmasked weights: {num_unmasked_weights}')
    
    if accuracy >= target_accuracy:
        best_ratio = ratio
        best_accuracy = accuracy

print(f'Highest pruning ratio within 1% of the original accuracy: {best_ratio}')
print(f'Accuracy at this pruning ratio: {best_accuracy:.2f}%')

import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
from torchvision.models import resnet18, ResNet18_Weights
import time
import numpy as np

# Function to evaluate the model and record latencies
def evaluate_model_with_latency(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    latencies = []
    with torch.no_grad():
        for data in dataloader:
            start_time = time.time()
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            end_time = time.time()
            latencies.append(end_time - start_time)
    accuracy = 100 * correct / total
    return accuracy, latencies

# Function to calculate p50 and p90 latencies
def calculate_p50_p90(latencies):
    latencies = np.array(latencies)
    p50 = np.percentile(latencies, 50)
    p90 = np.percentile(latencies, 90)
    return p50, p90

# Initialize the model and modify the final layer
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model.to(device)

# Load the trained model state dictionary
model.load_state_dict(torch.load('original_model.pth'))

# Evaluate the trained model and record latencies
original_accuracy, original_latencies = evaluate_model_with_latency(model, valloader)
print(f'Accuracy of the original model: {original_accuracy:.2f}%')
original_p50, original_p90 = calculate_p50_p90(original_latencies)
print(f'Original model p50 latency: {original_p50:.6f} seconds')
print(f'Original model p90 latency: {original_p90:.6f} seconds')

# Apply pruning with the optimal ratio (0.45)
pruning_ratio = best_ratio

# Create a new model instance and load the trained state dictionary
pruned_model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
num_ftrs = pruned_model.fc.in_features
pruned_model.fc = nn.Linear(num_ftrs, 10)
pruned_model.to(device)

pruned_model.load_state_dict(torch.load('original_model.pth'))  # Load the trained model

# Apply pruning
pruned_model = apply_pruning(pruned_model, pruning_ratio)

# Remove reparameterization
pruned_model = remove_pruning_reparameterization(pruned_model)

# Evaluate the pruned model and record latencies
pruned_accuracy, pruned_latencies = evaluate_model_with_latency(pruned_model, valloader)
print(f'Accuracy of the pruned model with ratio {pruning_ratio}: {pruned_accuracy:.2f}%')
pruned_p50, pruned_p90 = calculate_p50_p90(pruned_latencies)
print(f'Pruned model p50 latency: {pruned_p50:.6f} seconds')
print(f'Pruned model p90 latency: {pruned_p90:.6f} seconds')

# Compare the latencies
print("\nComparison:")
print(f'Original model p50 latency: {original_p50:.6f} seconds, p90 latency: {original_p90:.6f} seconds')
print(f'Pruned model p50 latency: {pruned_p50:.6f} seconds, p90 latency: {pruned_p90:.6f} seconds')

Accuracy of the original model: 85.16%
Pruning ratio: 0.25, Accuracy: 85.11%, Unmasked weights: 8379024.0
Pruning ratio: 0.26, Accuracy: 85.27%, Unmasked weights: 8267302.0
Pruning ratio: 0.27, Accuracy: 85.11%, Unmasked weights: 8155587.0
Pruning ratio: 0.28, Accuracy: 84.82%, Unmasked weights: 8043861.0
Pruning ratio: 0.29, Accuracy: 85.14%, Unmasked weights: 7932141.0
Pruning ratio: 0.3, Accuracy: 84.73%, Unmasked weights: 7820423.0
Pruning ratio: 0.31, Accuracy: 85.26%, Unmasked weights: 7708703.0
Pruning ratio: 0.32, Accuracy: 84.88%, Unmasked weights: 7596982.0
Pruning ratio: 0.33, Accuracy: 85.07%, Unmasked weights: 7485262.0
Pruning ratio: 0.33999999999999997, Accuracy: 84.70%, Unmasked weights: 7373540.0
Pruning ratio: 0.35, Accuracy: 84.67%, Unmasked weights: 7261821.0
Pruning ratio: 0.36, Accuracy: 85.01%, Unmasked weights: 7150100.0
Pruning ratio: 0.37, Accuracy: 84.56%, Unmasked weights: 7038377.0
Pruning ratio: 0.38, Accuracy: 84.76%, Unmasked weights: 6926663.0
Pruning r

## Conclusion


The objective of this experiment was to evaluate the effectiveness of pruning techniques on a ResNet-18 model trained on the CIFAR-10 dataset. The goal was to reduce the model size while maintaining its accuracy within 1% of the original model. 

Here are the key findings and conclusions from the experiment:

1. **Original Model Performance**:
    - **Accuracy**: The original ResNet-18 model achieved an accuracy of 85.38% on the validation set.
    - **Number of Parameters**: The original model had 11,181,642 parameters.
    - **Number of Unmasked Weights**: The original model had 11,181,642 unmasked weights, as it was not pruned.
    - **Latencies**:
        - **p50 Latency**: 0.009817 seconds
        - **p90 Latency**: 0.010340 seconds

2. **Pruning Experiment**:
    - Pruning was performed with various ratios ranging from 0.2 to 0.5.
    - The best pruning ratio that maintained the accuracy within 1% of the original model was found to be **0.44**.
    - At this pruning ratio, the pruned model achieved an accuracy of **84.47%**, which is within the 1% target accuracy threshold of the original model.

3. **Comparison of Unmasked Weights**:
    - **Original Model**: 11,181,642 unmasked weights.
    - **Pruned Model (0.44 Ratio)**: 6,256,337 unmasked weights.

4. **Latency Comparison**:
    - **Original Model**:
        - **p50 Latency**: 0.009817 seconds
        - **p90 Latency**: 0.010340 seconds
    - **Pruned Model (0.44 Ratio)**:
        - **p50 Latency**: 0.009822 seconds
        - **p90 Latency**: 0.010623 seconds

5. **Conclusions**:
    - The pruning technique successfully reduced the number of unmasked weights by approximately 44%, significantly decreasing the model's complexity.
    - The pruned model's accuracy was maintained within 1% of the original model's accuracy, demonstrating the effectiveness of the pruning process.
    - The latency analysis indicated that the pruned model had slightly higher p50 and p90 latencies compared to the original model. However, the increase in latency was minimal, showing that pruning did not substantially affect the model's inference speed.

Overall, the experiment demonstrates that pruning can effectively reduce the model size while maintaining high accuracy and acceptable latency. This technique is beneficial for deploying models in resource-constrained environments where model size and inference speed are critical factors.